**HASYv2 dataset**

Dataset obsahuje rukou psané matematické symboly. Skládá se z 168233 instancí v 369 třídách.
Autor datasetu ve své [práci](https://arxiv.org/pdf/1701.08380.pdf) uvádění maximální přesnost TF-CNN okolo 81%. Já se dostal s přesností na testovacích datech na 83% přesnosti. Vyšší přesnosti již téměř není možné dosáhnout kvůli množství snadno zaměnitelných znaků v datasetu (např. Ø vs ∅)

[Dataset](https://www.kaggle.com/martinthoma/hasyv2-dataset-friend-of-mnist) je volně dostupný na Kaggle.com.

Jako metriku úspěchu jsem zvolil přesnost, kde se pokouším o co nejvyšší procento.

In [ ]:
# Importy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.layers import Convolution2D, Dense, Dropout, Flatten, MaxPooling2D, Conv2D, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import cross_val_score

# Načtení csv obsahující cesty k jednotlivým souborům
data_labels = pd.read_csv("../input/hasyv2-dataset-friend-of-mnist/hasy-data-labels.csv")

In [ ]:
#Prozkoumání csv - používat budeme cesty souborů a cílový znak ve sloupci latex
data_labels.head()

In [ ]:
#Počet unikátních tříd
data_labels["latex"].unique().shape

In [ ]:
# Připravení trénovacích dat jako numpy pole vektorů z obrázků
from PIL import Image
from skimage import io, color
# tqdm slouží jen jako progress bar při načítání
from tqdm import tqdm_notebook as tqdm

base = "../input/hasyv2-dataset-friend-of-mnist/HASYv2/"
listOfSymbols = []
for items in tqdm(data_labels['path'].iteritems(), total=data_labels["path"].shape[0]): 
    pil_im = Image.open(base +items[1], 'r').convert('L')
    array1 = np.asarray(pil_im)
    listOfSymbols.append(array1)

In [ ]:
X_train_full = np.array(listOfSymbols)

In [ ]:
X_train_full.shape

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Zobrazení náhodného symbolu z trénovacích dat
some_symbol = X_train_full[2]
plt.imshow(some_symbol, cmap=matplotlib.cm.binary)

In [ ]:
#Jednotlivé třídy v listu
classes = data_labels["latex"].unique().tolist()
n_classes = len(classes)

In [ ]:
# Přípravení kategorických popisků pro klasifikaci
train_y = data_labels['latex']
listOfk = []
for items in data_labels['latex'].iteritems():
    intermediate = np.zeros(n_classes)
    index = classes.index(items[1])
    intermediate[index] = 1
    listOfk.append(intermediate)
categorical_labels = np.array(listOfk)    

In [ ]:
y_train_full = categorical_labels

In [ ]:
print(y_train_full.shape)

In [ ]:
#Zamíchání trénovacích dat pro testovací účely
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
X_train_full, y_train_full = unison_shuffled_copies(X_train_full, y_train_full)

In [ ]:
# Rozdělení na trénovací, testovací a validační podmnožiny
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
X_train.shape

In [ ]:
# Přidaní dimenze (instance, 32, 32, 1)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape

In [ ]:
#Vytvoření sekvenčního modelu, který na vstupu příjímá (32,32,1) a na výstupu má Dense vrstvu s n_classes = 369
model = Sequential()
model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(32,32,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Trénování modelu s optimizerem Adam, earlyStopping po 5 epochách bez pokroku
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss="categorical_crossentropy", optimizer=adam,
              metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[EarlyStopping(monitor="val_acc", min_delta=0.1, patience=5)])

In [ ]:
pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot()
plt.grid(True)

In [ ]:
import pandas as pd
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
plt.grid(True)

In [ ]:
# Evaluace modelu
score = model.evaluate(X_test, y_test)
print(f"Test accuarcy: {score[1] * 100:0.2f}%")

In [ ]:
# Vykreslení náhodného symbolu
index = 80
symbol = X_test[index]
plt.imshow(symbol, cmap=matplotlib.cm.binary)

In [ ]:
# Predicke symbolu
prediction = model.predict(symbol.reshape(-1,32,32,1))

In [ ]:
# Index nejvyšší pravděpodobnosti
class_prediction = np.argmax(prediction)
print(class_prediction)

In [ ]:
# Třída  predikce
classes[class_prediction]

In [ ]:
y_test[index]

In [ ]:
#Pořadí v y_test 
np.where(y_test[index]==1)[0][0]

**Závěr**

Pomocí konvoluční neuronové sítě se mi povedlo dosáhnout mírně lepších výsledků než autor datasetu úvádí ve své práci.